import pygame
from tkinter import *

pygame.init()
pygame.mixer.music.load('filename.wav')

'''This sets the size of the application window (currently a square)'''
size=(700, 700)
screen= pygame.display.set_mode(size)

'''This is assigning colors to names that are more user friendly'''
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (144, 144, 144)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

'''This helps to set the refresh rate'''
clock=pygame.time.Clock()
pygame.display.set_caption("Python Music Player - by ") #insert our names after the by, I'm not doing it yet as this is connected to a not-school account

'''This is for font of style Calibri at size 25 and is bold, no italics'''
font = pygame.font.SysFont('Calibri', 25, True, False)

class Track:
    """
    The "Track" object holds 
    """
    def __init__(self, title = "N/A", artist = "N/A", file = "error.wav"): # default values set
    self.title = title
    self.year = year
    self.file = file
    
    @property
    def title(self):
        """
        Function to return the track's title.
        """
        return self.__title
    
    @property
    def artist(self):
        """
        Function to return the track's artist.
        """
        return self.__artist
    
    @property
    def file(self):
        """
        Function to return the track's file.
        """
        return self.__file
    

def playTrack():
    """
    Function to play and resume a track.
    """
    pygame.mixer.music.play() # play for the first time, but
    pygame.mixer.music.unpause() # for each other timee

def pauseTrack():
    """
    Function to pause a track.
    """
    pygame.mixer.music.pause()

done=False
while not done:
    '''This sets the screen to default to the gray value I assigned earlier'''
    screen.fill(GRAY)
    pygame.display.flip()
    '''This sets the screen refresh rate to 60 times a second'''
    clock.tick(60)
    '''This draws a rectangle with a triangle shaped like a play button'''
    pygame.draw.rect(screen, GREEN, [350,550,100,100]) 
    pygame.draw.polygon(screen, WHITE, [[360,560], [360,640],[440,600]]) 
    pygame.display.flip()

    for event in pygame.event.get():
        # this isn't working as intended
        if event.type==pygame.QUIT:
            done=True
        elif event.type==pygame.KEYRIGHT:
            pygame.draw.rect(screen, GRAY, [100,100,300,100])
            text= font.render("This will make the music player skip to the next song", True, BLACK)
            screen.blit(text, [100,100])
            pygame.display.flip()
        elif event.type==pygame.KEYLEFT:
            if mixer.music.unpause()==False:
                pygame.draw.rect(screen, GRAY, [100,100,300,100])
                text= font.render("This will make the music player start at the beginning of the song", True, BLACK)
                screen.blit(text, [100,100])
                pygame.display.flip()
pygame.quit()
# while True: <-------- infinite loop to run our program
#
#
# (10/30)
# https://www.educative.io/answers/how-to-play-an-audio-file-in-pygame
# learn basic load, play, pause mixer functions
#
# (11/3) 
# yesterday in clas we learned how to make an object. we can use this for
# our movies. refer to Lab 10 for notes.

# this is what I'm using for learning graphics: http://programarcadegames.com/index.php?chapter=introduction_to_graphics
#
# add something like this somewhere to make sure the user wants to quit:
#            done_check=input("Are you sure you want to quit? (y for yes, n for no)": )
#            if done_check=="y" or done_check=="Y":
#                done=True
#            elif:
#                print("Returning back to the main menu...")